# Investigation of Gaussian Convolutions in N dimensional Space

Aims:
We propose a solution to the question of replay in the context of catastrophic forgetting. An attempt is made to create a 'net' to capture high-quallity 'reward' input vectors which contribute to learning. These vectors will be used in replay schemes.

For useful analysis to be made, there is a need for meaningful variation in input vector space. The current sampling technique uses a gaussian and thus forms an N dimensional sphere which is isotropic. This is not helpful for our analysis of finding high quality points. 

We attempt to 'mask' the gaussian using convolutions of other probability distributions. 

Examples include:
- Exponential
- Tent Function
- Uniform Dist (with skew)
- Parabolic

Import Libraries

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.special
import random
import torch

### Test for exponential convolved with gaussian.

In [ ]:
#analytic solution of Ex-Gauss dist:
def convolve_exp_norm(alpha, mu, sigma, x):
    """
    Parameters:
    alpha = index of exponential
    mu = mean of the Gaussian (i.e. the centroid). If none-zero it is an offset
    sigma = variance of normal distribution
    x = input value


    """
    co = alpha/2.0 * np.exp( alpha*mu+ alpha*alpha*sigma*sigma/2.0)
    x_erf = (mu + alpha*sigma*sigma - x)/(np.sqrt(2.0)*sigma)
    y = co * np.exp(-alpha*x) * (1.0 - scipy.special.erf(x_erf))
    return y


#randomly sample parameters
#Debate between using gaussian vs uniform random dist

Using a gaussian to randomly sample the parameter values for the convolution will make sure our input distributions are mostly similar and we can highlight the rare events. However, using a uniform dist will allow us to have an unbiased view of which inputs are best...

In [ ]:
#Uniform dist
alpha = 